In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import time  # Import for adding delay

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Load corner points calibration data
corner_points = np.load('corner_points.npz')['corner_points']
tl, tr, bl, br = corner_points[:4]

# Define class names (must match your trained model)
class_names = ['AA', 'D type', '9v']

# Initialize camera (adjust camera_index as needed)
camera_index = 1  # Example: change this to match your external camera index
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    exit()

# Function to map detected position to normalized grid coordinates
def map_to_grid(x, y, tl, tr, bl, br):
    # Compute vectors for mapping to normalized grid coordinates
    v1 = np.array([tr[0] - tl[0], tr[1] - tl[1]])
    v2 = np.array([bl[0] - tl[0], bl[1] - tl[1]])
    vp = np.array([x - tl[0], y - tl[1]])

    # Calculate normalized grid coordinates (u, v)
    u = np.dot(vp, v1) / np.dot(v1, v1)
    v = np.dot(vp, v2) / np.dot(v2, v2)

    return u, v

# Function to convert grid coordinates to robot coordinates
def grid_to_robot(u, v, tl, tr, bl, br):
    # Calculate the robot coordinates (X, Y, Z) from the normalized grid coordinates (u, v)
    x_robot = tl[0] + u * (tr[0] - tl[0]) + v * (bl[0] - tl[0])
    y_robot = tl[1] + u * (tr[1] - tl[1]) + v * (bl[1] - tl[1])
    z_robot = tl[2] + u * (tr[2] - tl[2]) + v * (bl[2] - tl[2])  # Adjust for height if necessary

    return x_robot, y_robot, z_robot

# Function to calculate X2, Y2, Z
def calculate_X2_Y2(x1, y1, z1):
    X2 = 0.71 * y1 + 351.32
    Y2 = 0.5381 * x1 - 0.1054 * y1 + 78
    Z2 = z1  # Z2 is the same as Z1
    return int(X2), int(Y2), int(Z2)  # Convert to integers

try:
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame

        if not ret:
            print("Error: Failed to capture image")
            break

        # Perform inference
        results = model(frame)[0]  # Get the first result

        # Iterate through the detected objects
        detections = []
        for result in results.boxes.data:
            x1, y1, x2, y2, confidence, class_id = result.tolist()

            # Calculate center coordinates of the bounding box
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            # Map the detected position to the grid coordinates
            u, v = map_to_grid(x_center, y_center, tl, tr, bl, br)

            # Convert grid coordinates to robot coordinates
            x_robot, y_robot, z_robot = grid_to_robot(u, v, tl, tr, bl, br)

            # Calculate new X2, Y2, and Z based on the formula provided
            new_X2, new_Y2, new_Z = calculate_X2_Y2(x_center, y_center, z_robot)

            # Save detection information (battery type and new coordinates)
            detections.append({
                'battery_type': class_names[int(class_id)],
                'new_coordinates': (new_X2, new_Y2, new_Z)
            })

            # Draw bounding box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            label = f'{class_names[int(class_id)]} {confidence:.2f}'
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Display the new coordinates on the frame
            coord_text = f'X2: {new_X2}, Y2: {new_Y2}, Z: {new_Z}'
            cv2.putText(frame, coord_text, (int(x1), int(y2 + 20)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the frame with detections and coordinates
        cv2.imshow('Battery Detection', frame)

        # Print or log the detected battery information
        for detection in detections:
            print(f"Detected {detection['battery_type']} at new coordinates {detection['new_coordinates']}")

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Delay of 20 seconds
        time.sleep(20)

except KeyboardInterrupt:
    print("Detection stopped by user.")
finally:
    # Release the capture and close all windows
    cap.release()
    cv2.destroyAllWindows()



0: 480x640 (no detections), 261.2ms
Speed: 5.3ms preprocess, 261.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 191.7ms
Speed: 4.0ms preprocess, 191.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.2ms
Speed: 3.6ms preprocess, 201.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 D type, 174.1ms
Speed: 3.0ms preprocess, 174.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at new coordinates (518, 274, 162)

0: 480x640 1 D type, 190.0ms
Speed: 4.6ms preprocess, 190.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at new coordinates (502, 251, 162)

0: 480x640 1 AA, 1 D type, 202.1ms
Speed: 3.4ms preprocess, 202.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected AA at new coordinates (548, 268, 162)
Detected D type at new coordinates (548, 268, 162)

0: 4

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Load corner points calibration data
corner_points = np.load('corner_points.npz')['corner_points']
tl, tr, bl, br = corner_points[:4]

# Define class names (must match your trained model)
class_names = ['AA', 'D type', '9v']

# Initialize camera (adjust camera_index as needed)
camera_index = 1  # Example: change this to match your external camera index
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    exit()

# Function to map detected position to normalized grid coordinates
def map_to_grid(x, y, tl, tr, bl, br):
    # Compute vectors for mapping to normalized grid coordinates
    v1 = np.array([tr[0] - tl[0], tr[1] - tl[1]])
    v2 = np.array([bl[0] - tl[0], bl[1] - tl[1]])
    vp = np.array([x - tl[0], y - tl[1]])

    # Calculate normalized grid coordinates (u, v)
    u = np.dot(vp, v1) / np.dot(v1, v1)
    v = np.dot(vp, v2) / np.dot(v2, v2)

    return u, v

# Function to convert grid coordinates to robot coordinates
def grid_to_robot(u, v, tl, tr, bl, br):
    # Calculate the robot coordinates (X, Y, Z) from the normalized grid coordinates (u, v)
    x_robot = tl[0] + u * (tr[0] - tl[0]) + v * (bl[0] - tl[0])
    y_robot = tl[1] + u * (tr[1] - tl[1]) + v * (bl[1] - tl[1])
    z_robot = tl[2] + u * (tr[2] - tl[2]) + v * (bl[2] - tl[2])  # Adjust for height if necessary

    return x_robot, y_robot, z_robot

# Function to calculate X2, Y2, Z
def calculate_X2_Y2(x1, y1, z1):
    X2 = 0.71 * y1 + 351.32
    Y2 = 0.5381 * x1 - 0.1054 * y1 + 78
    Z2 = z1  # Z2 is the same as Z1
    return int(X2), int(Y2), int(Z2)  # Convert to integers

try:
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame

        if not ret:
            print("Error: Failed to capture image")
            break

        # Display the frame without detection
        cv2.imshow('Battery Detection', frame)

        # Wait for spacebar ('space') press to trigger detection
        if cv2.waitKey(1) & 0xFF == ord(' '):
            # Perform inference when spacebar is pressed
            results = model(frame)[0]  # Get the first result

            # Iterate through the detected objects
            detections = []
            for result in results.boxes.data:
                x1, y1, x2, y2, confidence, class_id = result.tolist()

                # Calculate center coordinates of the bounding box
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2

                # Map the detected position to the grid coordinates
                u, v = map_to_grid(x_center, y_center, tl, tr, bl, br)

                # Convert grid coordinates to robot coordinates
                x_robot, y_robot, z_robot = grid_to_robot(u, v, tl, tr, bl, br)

                # Calculate new X2, Y2, and Z based on the formula provided
                new_X2, new_Y2, new_Z = calculate_X2_Y2(x_center, y_center, z_robot)

                # Save detection information (battery type and new coordinates)
                detections.append({
                    'battery_type': class_names[int(class_id)],
                    'new_coordinates': (new_X2, new_Y2, new_Z)
                })

                # Draw bounding box and label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                label = f'{class_names[int(class_id)]} {confidence:.2f}'
                cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                # Display the new coordinates on the frame
                coord_text = f'X2: {new_X2}, Y2: {new_Y2}, Z: {new_Z}'
                cv2.putText(frame, coord_text, (int(x1), int(y2 + 20)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the frame with detections and coordinates
            cv2.imshow('Battery Detection', frame)

            # Print or log the detected battery information
            for detection in detections:
                print(f"Detected {detection['battery_type']} at new coordinates {detection['new_coordinates']}")

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detection stopped by user.")
finally:
    # Release the capture and close all windows
    cap.release()
    cv2.destroyAllWindows()



0: 480x640 1 9v, 288.4ms
Speed: 6.0ms preprocess, 288.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected 9v at new coordinates (551, 210, 162)

0: 480x640 1 9v, 273.4ms
Speed: 3.4ms preprocess, 273.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected 9v at new coordinates (550, 210, 162)

0: 480x640 1 9v, 271.3ms
Speed: 3.0ms preprocess, 271.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected 9v at new coordinates (550, 210, 162)

0: 480x640 1 D type, 274.6ms
Speed: 3.1ms preprocess, 274.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at new coordinates (492, 226, 162)

0: 480x640 1 D type, 284.3ms
Speed: 5.0ms preprocess, 284.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detected D type at new coordinates (492, 226, 162)

0: 480x640 1 D type, 268.2ms
Speed: 4.0ms preprocess, 268.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
D

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Load corner points calibration data
corner_points = np.load('corner_points.npz')['corner_points']
tl, tr, bl, br = corner_points[:4]

# Define class names (must match your trained model)
class_names = ['AA', 'D type', '9v']

# Initialize camera (adjust camera_index as needed)
camera_index = 1  # Example: change this to match your external camera index
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    exit()

# Function to map detected position to normalized grid coordinates
def map_to_grid(x, y, tl, tr, bl, br):
    v1 = np.array([tr[0] - tl[0], tr[1] - tl[1]])
    v2 = np.array([bl[0] - tl[0], bl[1] - tl[1]])
    vp = np.array([x - tl[0], y - tl[1]])

    u = np.dot(vp, v1) / np.dot(v1, v1)
    v = np.dot(vp, v2) / np.dot(v2, v2)

    return u, v

# Function to convert grid coordinates to robot coordinates
def grid_to_robot(u, v, tl, tr, bl, br):
    x_robot = tl[0] + u * (tr[0] - tl[0]) + v * (bl[0] - tl[0])
    y_robot = tl[1] + u * (tr[1] - tl[1]) + v * (bl[1] - tl[1])
    z_robot = tl[2] + u * (tr[2] - tl[2]) + v * (bl[2] - tl[2])
    return x_robot, y_robot, z_robot

# Function to calculate X2, Y2, Z
def calculate_X2_Y2(x1, y1, z1):
    X2 = 0.71 * y1 + 351.32
    Y2 = 0.5381 * x1 - 0.1054 * y1 + 78
    Z2 = z1
    return int(X2), int(Y2), int(Z2)

try:
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame

        if not ret:
            print("Error: Failed to capture image")
            break

        # Display the frame without detection
        cv2.imshow('Battery Detection', frame)

        # Wait for spacebar ('space') press to trigger detection
        if cv2.waitKey(1) & 0xFF == ord(' '):
            # Perform inference when spacebar is pressed
            results = model(frame)[0]  # Get the first result

            # Iterate through the detected objects
            detections = []
            for result in results.boxes.data:
                x1, y1, x2, y2, confidence, class_id = result.tolist()

                # Calculate center coordinates of the bounding box
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2

                # Map the detected position to the grid coordinates
                u, v = map_to_grid(x_center, y_center, tl, tr, bl, br)

                # Convert grid coordinates to robot coordinates
                x_robot, y_robot, z_robot = grid_to_robot(u, v, tl, tr, bl, br)

                # Calculate new X2, Y2, and Z based on the formula provided
                new_X2, new_Y2, new_Z = calculate_X2_Y2(x_center, y_center, z_robot)

                # Save detection information (battery type and new coordinates)
                detections.append({
                    'battery_type': class_names[int(class_id)],
                    'new_coordinates': (new_X2, new_Y2, new_Z)
                })

                # Draw bounding box and label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                label = f'{class_names[int(class_id)]} {confidence:.2f}'
                cv2.putText(frame, label, (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                # Display the new coordinates on the frame
                coord_text = f'X2: {new_X2}, Y2: {new_Y2}, Z: {new_Z}'
                cv2.putText(frame, coord_text, (int(x1), int(y2) + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the frame with detections and coordinates
            cv2.imshow('Battery Detection', frame)

            # Print or log the detected battery information
            for detection in detections:
                print(f"Detected {detection['battery_type']} at new coordinates {detection['new_coordinates']}")

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detection stopped by user.")
finally:
    # Release the capture and close all windows
    cap.release()
    cv2.destroyAllWindows()


Detection stopped by user.
